In [1]:
import pandas as pd
import requests
import os 
import ast
import time
from time import sleep
import logging
from pydantic import RootModel, conint
from typing import Tuple

In [2]:
system_prompt = """
    You are a helpful literary assistant, helping me with my master thesis on gender in literature. 
    Your job is to search the internet and assess whether a book features a **female major character** (fmc).
    
   You MUST ALWAYS return your response as a Python list with exactly two elements: [fmc_present, fmc_name]
    - `fmc_present` = 1 if yes, 0 if no
    - `fmc_name` = the most important female major character's name, or an empty string.

    Never return anything else. No explanations, no full sentences. Just the list.

    Examples:
    - For "To Kill A Mockingbird" by Harper Lee (has a fmc): [1, 'Scout Finch']
    - For "The Hobbit and The Lord of the Rings" by J.R.R. Tolkien (doesn't have a fmc): [0, '']

    The female major character does not need to be the protagonist, but she must be a central presence in the narrative, 
    with **more significance than a secondary or background character**.

    A female character is considered a **major character** if she plays a substantial role in the story, 
    regularly appears throughout the book, and if her role significantly contributes to relationships or events that 
    shape the plot or other characters.

    Characters who serve only as symbolic figures, plot devices, or one-time motivators without meaningful presence 
    or development should not be considered major characters.

    Examples of female major characters:
    - **Katniss Everdeen** in **The Hunger Games** – protagonist, central to the plot, her choices drive the rebellion and story progression.
    – **Hermione Granger** in *Harry Potter* – not the protagonist, but central to plot progression and character development.
    
    Examples of female NON-major characters:
    - **Prim Everdeen** in **The Hunger Games** – important to Katniss's emotional motivation, but doesn’t influence the plot or have significant character development beyond being a catalyst.
    - **Professor McGonagall** in **Harry Potter** – important supporting character, but doesn’t influence the main plot or have significant character development beyond being a mentor and guide to Harry and his friends.
    """

user_prompt_template = "Is there a female major character in the book {title} by {author}?"

In [3]:
# Specify answer format

# Define the tuple type: (0 or 1, followed by a string)
MyFormat = Tuple[conint(ge=0, le=1), str]

# Use RootModel to define the model
class AnswerFormat(RootModel[MyFormat]):
    pass

In [4]:
# Load all books
df_all = pd.read_csv("books.csv")
df_all = df_all.drop(columns=['genres'])
df_all = df_all.drop_duplicates(subset=["title", "author"]).reset_index(drop=True)
df_all = df_all.dropna(subset=["title", "author"])
df_all["author"] = df_all["author"].str.replace("*", "", regex=False).str.strip()
df_all

,title,author
0,Jackie Me Baseball Card Adventure 2,"Gutman, Dan"
1,Honus Me A Baseball Card Adventure 1,"Gutman, Dan"
2,Wildflower Hill,"Freeman, Kimberley"
3,Oracle Night,"Auster, Paul"
4,As a Driven Leaf,"Steinberg, Milton"
...,...,...
17999,Truly Madly Deeply,"Kazi, Faraaz"
18000,Sloop of War Richard Bolitho 6,"Kent, Alexander"
18001,Purification Autumn 3,"Moody, David"
18002,The American Heiress,"Goodwin, Daisy"


In [5]:
# Split big all_books up into smaller parts for less risk 
# in 60 minutes, I can run around 1,525 books

# First try for 2 hours, so around 3,050 books
# df_subset_1 = df_all.iloc[:3050].copy()
# df_subset_1

# Create second subset
df_subset_2 = df_all.iloc[3050:14000].copy()
df_subset_2

,title,author
3050,The Business of Dying Dennis Milne 1,"Kernick, Simon"
3051,Stamboul Train,"Greene, Graham"
3052,Marjorie Morningstar,"Wouk, Herman"
3053,Devices and Desires Engineer Trilogy 1,"Parker, K.J."
3054,Turks Fruit,"Wolkers, Jan"
...,...,...
13995,A Fire in the Flesh Flesh and Fire 3,"Armentrout, Jennifer L."
13996,The Last Tale of the Flower Bride,"Chokshi, Roshani"
13997,Sworn to Silence Kate Burkholder 1,"Castillo, Linda"
13998,Boom,"Haddon, Mark"


In [6]:
# Set up logging for error tracking
logging.basicConfig(filename='subset2_log.log', level=logging.INFO)

# Setting API key
YOUR_API_KEY = os.environ["PERPLEXITY_API_KEY"]

# Headers for the API request
headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json"
}

# Endpoint for the API
url = "https://api.perplexity.ai/chat/completions"

# Store responses, citations, and titles + authors for progress
responses = []
citations = []
titles = []
authors = []

In [9]:
# Function to send a request to the API
def send_request(title, author, retries=3):
    user_prompt = user_prompt_template.format(title=title, author=author)
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 200,
        "temperature": 0,  
        "response_format": {
            "type": "json_schema",
            "json_schema": {"schema": AnswerFormat.model_json_schema()},
        },
    }

    try:
        # Make the request to the API
        response = requests.post(url, json=payload, headers=headers)
        
        # Check if the response is successful
        if response.status_code == 200:
            response_json = response.json()
            answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')
            citation = response_json.get("citations", [])
            return answer, citation
        else:
            raise Exception(f"API call failed with status code {response.status_code}")
    
    except Exception as e:
        # Log the error and retry if retries left
        logging.error(f"Error with request for {title} by {author}: {str(e)}")
        if retries > 0:
            logging.info(f"Retrying {retries} more times...")
            sleep(60)  # Sleep to avoid hitting rate limits
            return send_request(title, author, retries - 1)  # Retry the request
        else:
            logging.error(f"Failed after retries: {title} by {author}")
            return f"Error: {str(e)}", []

In [11]:
def save_progress():
    if titles:  # Only save if there's something new
        progress_df = pd.DataFrame({
            'title': titles,
            'author': authors,
            'answer': responses,
            'citation': citations
        })

        file_exists = os.path.exists('progress_fmc.csv')

        if file_exists:
            progress_df.to_csv('progress_fmc.csv', mode='a', header=False, index=False)
            logging.info(f"Progress appended: {len(titles)} books.")
        else:
            progress_df.to_csv('progress_fmc.csv', index=False)
            logging.info(f"Progress saved: {len(titles)} books.")

        # Clear the lists so next save only writes new data
        titles.clear()
        authors.clear()
        responses.clear()
        citations.clear()

In [12]:
start_time = time.time()

# Run for df_subset_1
for index, row in df_test.iterrows():
    title = row["title"]
    author = row["author"]
    
    # Send request and handle response
    answer, citation = send_request(title, author)
    
    # Store responses
    responses.append(answer)
    citations.append(citation)
    titles.append(title)
    authors.append(author)

    # Save progress every 10 books (adjust as necessary)
    if (index + 1) % 500 == 0:
        save_progress()

# Save the final progress
save_progress()

# Log the total execution time
end_time = time.time()
execution_time = end_time - start_time
logging.info(f"Completed 3050 book requests in {execution_time:.2f} seconds.")

In [13]:
test = pd.read_csv('progress_fmc.csv')
test

,title,author,answer,citation
0,The Business of Dying Dennis Milne 1,"Kernick, Simon","[0, """"]",['https://en.wikipedia.org/wiki/The_Business_o...
1,Stamboul Train,"Greene, Graham","[1, ""Coral Musker""]",['https://astrofella.wordpress.com/2014/09/25/...
2,Marjorie Morningstar,"Wouk, Herman","[1, ""Marjorie Morgenstern""]",['https://en.wikipedia.org/wiki/Marjorie_Morni...
3,Devices and Desires Engineer Trilogy 1,"Parker, K.J.","[0, """"]",['https://theidlewoman.net/2017/04/07/devices-...
4,Turks Fruit,"Wolkers, Jan","[1, ""Olga""]",['https://en.wikipedia.org/wiki/Turks_Fruit_(n...
5,The Cellar,"Laymon, Richard","[1, ""Donna""]",['https://www.goodreads.com/book/show/3206436'...
6,Among the Missing,"Laymon, Richard","[1, ""Mary 'Pac' Hodges""]",['https://en.wikipedia.org/wiki/Among_the_Miss...
7,Drama Queen Candy Apple 5,"Bergen, Lara","[1, ""Charlie""]",['https://www.goodreads.com/book/show/1452286'...
8,The Mailbox,"Shafer, Audrey","[0, """"]",['https://bfgb.wordpress.com/2009/12/07/the-ma...
9,How to Be a Girly Girl in Just Ten Days Candy ...,"Papademetriou, Lisa","[1, ""Nicolette Spicer""]",['https://beautyandthebeanboots.wordpress.com/...


$61.97 to $45.21
-> $16.76

3050 -> $16.76
18,004 -> 98,93  (*5.903)
